In [1]:
import numpy
import numpy as np
import pandas as pd
import sys, glob
import urllib3
import h5py
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
warnings.filterwarnings('ignore')
import pycountry
sys.path.append(os.getcwd()+'/../../cds-backend/code/')
import cds_eua3 as eua
# import numbaprocess
import copy
import glob
from numba import njit
import pandas
import glob
import pickle
import matplotlib
import matplotlib.pyplot as plt
import h5py as h5


# load data for a list of stations (IGRA, RHARM)
# stations = ['USM00070316' , 'USM00070350', 'USM00070308', 'PSM00091408']

### TO DO
# Here: select the name of the download directory according  to TEST or PUB
# You must change the API file .cdsapi accordingly before the download !!!

cds  = '_test'
cds = ''
out_dir =  'download' + cds


In [2]:
def request(rqdict, source, cds='_test', remove_file=True , outdir = 'download'):
    """
    Remember to UPDATE the file .cdsapirc on the server home directory with the CORRECT cds address!
    See e.g. https://cds-test.copernicus-climate.eu/cdsapp#!/dataset/insitu-observations-igra-baseline-network?tab=form 
    """
    if cds != '_test':
        cds = ''
    t0 = time.time()

    c = cdsapi.Client()
    print(c)
    r = c.retrieve(
        source, rqdict)
    print('Request took: ' + str(time.time() - t0) + ' seconds')
    if True:
        r.download(target='download.zip')
        assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    z = zipfile.ZipFile('download.zip')
    
    z.extractall(path='./' + outdir + '/' )
    z.close()
    return

def wrapper_request_date_station(date='2009-04-01', station='PSM00091408', outdir = 'download'):

    out = request({
            'source': 'IGRA_H',# 'IGRA'
            'variable':['air_temperature'],
            'year':date.split('-')[0], 'month':date.split('-')[1] , 'day':date.split('-')[2],
            'format': 'csv-lev.zip',
            'station_name': station,
            'archive_type': 'harmonized_global_radiosonde_archive',

            },igra, cds=cds, remove_file=False , outdir = outdir)

    files = glob.glob(outdir+'/*.csv')

    file_dest = outdir + '/' + station + '_RHARM_data.csv'

    print('FILES HARM' , files , '  ' , outdir)
    for f in files:
        
        if '.csv' in f and 'data' not in f and 'IGRA_H_' in f:
            os.rename( f, file_dest )

    # reading the data as pandas dataframe
    df = pd.read_csv(file_dest, skiprows=14)
    # storing only standard pressure levels
    df = df[df.air_pressure.isin(stdplevs ) ]
    df =df.sort_values(by=['report_timestamp','air_pressure'])
    # da = eua.CDMDataset('download/USM00070308_IGRA_data.nc').to_dataframe() ### example
    all_HARM_df[station] = df 

    out = request({
    'source': 'IGRA',# 'IGRA'
    'variable':['air_temperature'],
    'year':date.split('-')[0], 'month':date.split('-')[1] , 'day':date.split('-')[2],
    'format': 'csv-lev.zip',
    'station_name': station,
    'archive_type': 'global_radiosonde_archive',

    },igra, cds=cds, remove_file=False, outdir = outdir )

    filess = glob.glob(outdir+'/*.csv')
    print('FILES IGRA' ,  files , '  ' , outdir)
    file_des = outdir + '/' + station + '_IGRA_data.csv'
    for ff in filess:
        if '.csv' in ff and 'data' not in ff and 'HARM' not in ff :
            os.rename( ff, file_des )

    dd = pd.read_csv(file_des, skiprows=14)
    dd = dd[dd.air_pressure.isin(stdplevs ) ] 
    dd = dd.sort_values(by=['report_timestamp','air_pressure'])

    all_IGRA_df[station] = dd        
    
igra = 'insitu-observations-igra-baseline-network'



## TESTS to do

Now to make your test I suggest working on one of these two points (randomly chosen):

- date_of_observation,press,temp,temp_h,err_temp_h,idstation,latitude,longitude

- 2009-04-01 12:00:00+00,10000.0,192.95,192.16,0.63,PSM00091408,7.3333,134.4833

- 2009-04-21 12:00:00+00,10000.0,218.05,217.34,0.68,USM00070308,57.15,-170.2167

For station PSM00091408, I also successfully checked the presence of values at the significant levels (you refer to them as interpolated levels although 10000 Pa is a mandatory level, not interpolated in RHARM) and at 10000 Pa as well.

In [3]:
#stations = ['USM00070316' , 'USM00070350', 'USM00070308', 'PSM00091408']
stdplevs = [ i*100 for i in [10.0, 20.0, 30.0, 50.0, 70.0, 100.0, 150.0, 200.0, 250.0, 300.0, 400.0, 500.0, 700.0, 850.0, 925.0, 1000.0] ]
cds = 'test'
out_dir =  'download_single' + cds

# Containers for dataframes
all_HARM_df = {} 
all_IGRA_df = {}


## READ PSM00091408 2009-04-01

In [4]:
dummy = wrapper_request_date_station(date='2009-04-01', station='PSM00091408', outdir = out_dir )

2022-08-29 12:54:31,091 INFO Welcome to the CDS
2022-08-29 12:54:31,093 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network


2022-08-29 12:54:31,215 INFO Request is completed
2022-08-29 12:54:31,217 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data3/adaptor.insitu_reference.retrieve-1661763875.3544014-4789-12-d031a3f3-bdd8-436b-b194-4f099325851e.zip to download.zip (2.2K)


Request took: 0.3076300621032715 seconds


2022-08-29 12:54:31,389 INFO Download rate 12.8K/s
2022-08-29 12:54:31,480 INFO Welcome to the CDS
2022-08-29 12:54:31,481 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2022-08-29 12:54:31,593 INFO Request is completed
2022-08-29 12:54:31,595 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data8/adaptor.insitu_reference.retrieve-1661764581.3565335-31204-16-6c180efc-5874-4b7c-bb43-956fbedc9a5c.zip to download.zip (2.3K)


FILES HARM ['download_singletest/PSM00091408_RHARM_data.csv', 'download_singletest/PSM00091408_IGRA_data.csv', 'download_singletest/USM00070308_RHARM_data.csv', 'download_singletest/USM00070308_IGRA_data.csv', 'download_singletest/IGRA_H_20090401_20090401_global_cdm-levv1.csv']    download_singletest
Request took: 0.14530587196350098 seconds


2022-08-29 12:54:31,767 INFO Download rate 13.3K/s


FILES IGRA ['download_singletest/PSM00091408_RHARM_data.csv', 'download_singletest/PSM00091408_IGRA_data.csv', 'download_singletest/USM00070308_RHARM_data.csv', 'download_singletest/USM00070308_IGRA_data.csv', 'download_singletest/IGRA_H_20090401_20090401_global_cdm-levv1.csv']    download_singletest


## READ USM00070308 2009-04-21

In [5]:
dummy = wrapper_request_date_station(date='2009-04-21', station='USM00070308', outdir = out_dir )

2022-08-29 12:54:31,857 INFO Welcome to the CDS
2022-08-29 12:54:31,858 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network


2022-08-29 12:54:32,467 INFO Request is completed
2022-08-29 12:54:32,469 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data7/adaptor.insitu_reference.retrieve-1661764593.1514716-25499-19-1cbcf5df-ae35-40b2-b712-f96b620e81a5.zip to download.zip (1.6K)


Request took: 0.6428325176239014 seconds


2022-08-29 12:54:32,613 INFO Download rate 11.2K/s


FILES HARM ['download_singletest/PSM00091408_RHARM_data.csv', 'download_singletest/PSM00091408_IGRA_data.csv', 'download_singletest/USM00070308_RHARM_data.csv', 'download_singletest/USM00070308_IGRA_data.csv', 'download_singletest/IGRA_H_20090421_20090421_global_cdm-levv1.csv']    download_singletest


2022-08-29 12:54:32,695 INFO Welcome to the CDS
2022-08-29 12:54:32,696 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2022-08-29 12:54:32,834 INFO Request is completed
2022-08-29 12:54:32,835 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data8/adaptor.insitu_reference.retrieve-1661764594.855593-524-17-ec4d38cd-aa43-4bc9-aa3f-64f38ca3c63d.zip to download.zip (1.6K)


Request took: 0.17077112197875977 seconds


2022-08-29 12:54:33,987 INFO Download rate 1.4K/s


FILES IGRA ['download_singletest/PSM00091408_RHARM_data.csv', 'download_singletest/PSM00091408_IGRA_data.csv', 'download_singletest/USM00070308_RHARM_data.csv', 'download_singletest/USM00070308_IGRA_data.csv', 'download_singletest/IGRA_H_20090421_20090421_global_cdm-levv1.csv']    download_singletest


## RESULTS PSM00091408

In [6]:
#all_IGRA_df.keys()

In [7]:
all_IGRA_df['PSM00091408']

,station_name,report_timestamp,report_id,longitude,latitude,air_pressure,air_temperature
172,PSM00091408,2009-04-01 00:00:00+00,32128157,134.4833,7.3333,1000.0,233.45
162,PSM00091408,2009-04-01 00:00:00+00,32128157,134.4833,7.3333,2000.0,220.85
158,PSM00091408,2009-04-01 00:00:00+00,32128157,134.4833,7.3333,3000.0,213.75
152,PSM00091408,2009-04-01 00:00:00+00,32128157,134.4833,7.3333,5000.0,209.35
146,PSM00091408,2009-04-01 00:00:00+00,32128157,134.4833,7.3333,7000.0,196.65
139,PSM00091408,2009-04-01 00:00:00+00,32128157,134.4833,7.3333,10000.0,189.55
137,PSM00091408,2009-04-01 00:00:00+00,32128157,134.4833,7.3333,15000.0,206.35
134,PSM00091408,2009-04-01 00:00:00+00,32128157,134.4833,7.3333,20000.0,219.45
131,PSM00091408,2009-04-01 00:00:00+00,32128157,134.4833,7.3333,25000.0,231.75
126,PSM00091408,2009-04-01 00:00:00+00,32128157,134.4833,7.3333,30000.0,242.05


In [8]:
all_HARM_df['PSM00091408']

,station_name,radiosonde_code,sensor_model,report_timestamp,actual_time,report_id,longitude,latitude,height_of_station_above_sea_level,air_pressure,air_temperature
99,PSM00091408,52,Vaisala RS80-57H,2009-04-01 00:00:00+00,2009-03-31 23:03:00+00,1963813,134.483,7.3333,30,1000,NaN
89,PSM00091408,52,Vaisala RS80-57H,2009-04-01 00:00:00+00,2009-03-31 23:03:00+00,1963813,134.483,7.3333,30,2000,NaN
85,PSM00091408,52,Vaisala RS80-57H,2009-04-01 00:00:00+00,2009-03-31 23:03:00+00,1963813,134.483,7.3333,30,3000,NaN
79,PSM00091408,52,Vaisala RS80-57H,2009-04-01 00:00:00+00,2009-03-31 23:03:00+00,1963813,134.483,7.3333,30,5000,NaN
73,PSM00091408,52,Vaisala RS80-57H,2009-04-01 00:00:00+00,2009-03-31 23:03:00+00,1963813,134.483,7.3333,30,7000,NaN
66,PSM00091408,52,Vaisala RS80-57H,2009-04-01 00:00:00+00,2009-03-31 23:03:00+00,1963813,134.483,7.3333,30,10000,NaN
64,PSM00091408,52,Vaisala RS80-57H,2009-04-01 00:00:00+00,2009-03-31 23:03:00+00,1963813,134.483,7.3333,30,15000,NaN
61,PSM00091408,52,Vaisala RS80-57H,2009-04-01 00:00:00+00,2009-03-31 23:03:00+00,1963813,134.483,7.3333,30,20000,NaN
58,PSM00091408,52,Vaisala RS80-57H,2009-04-01 00:00:00+00,2009-03-31 23:03:00+00,1963813,134.483,7.3333,30,25000,NaN
53,PSM00091408,52,Vaisala RS80-57H,2009-04-01 00:00:00+00,2009-03-31 23:03:00+00,1963813,134.483,7.3333,30,30000,NaN


## RESULTS USM00070308

In [9]:
all_IGRA_df['USM00070308']

,station_name,report_timestamp,report_id,longitude,latitude,air_pressure,air_temperature
115,USM00070308,2009-04-21 00:00:00+00,31248324,-170.2167,57.15,1000.0,234.35
109,USM00070308,2009-04-21 00:00:00+00,31248324,-170.2167,57.15,2000.0,227.55
105,USM00070308,2009-04-21 00:00:00+00,31248324,-170.2167,57.15,3000.0,224.35
101,USM00070308,2009-04-21 00:00:00+00,31248324,-170.2167,57.15,5000.0,221.45
98,USM00070308,2009-04-21 00:00:00+00,31248324,-170.2167,57.15,7000.0,221.05
93,USM00070308,2009-04-21 00:00:00+00,31248324,-170.2167,57.15,10000.0,220.85
85,USM00070308,2009-04-21 00:00:00+00,31248324,-170.2167,57.15,15000.0,217.05
76,USM00070308,2009-04-21 00:00:00+00,31248324,-170.2167,57.15,20000.0,210.75
71,USM00070308,2009-04-21 00:00:00+00,31248324,-170.2167,57.15,25000.0,214.05
69,USM00070308,2009-04-21 00:00:00+00,31248324,-170.2167,57.15,30000.0,223.85


In [10]:
all_HARM_df['USM00070308']

,station_name,radiosonde_code,sensor_model,report_timestamp,actual_time,report_id,longitude,latitude,height_of_station_above_sea_level,air_pressure,air_temperature
73,USM00070308,51,VIZ-B2 (United States),2009-04-21 00:00:00+00,2009-04-20 23:09:00+00,11265784,-170.217,57.15,10,1000,234.35
67,USM00070308,51,VIZ-B2 (United States),2009-04-21 00:00:00+00,2009-04-20 23:09:00+00,11265784,-170.217,57.15,10,2000,227.55
63,USM00070308,51,VIZ-B2 (United States),2009-04-21 00:00:00+00,2009-04-20 23:09:00+00,11265784,-170.217,57.15,10,3000,224.35
74,USM00070308,51,VIZ-B2 (United States),2009-04-21 00:00:00+00,2009-04-20 23:09:00+00,11265784,-170.217,57.15,10,5000,221.45
57,USM00070308,51,VIZ-B2 (United States),2009-04-21 00:00:00+00,2009-04-20 23:09:00+00,11265784,-170.217,57.15,10,7000,221.05
52,USM00070308,51,VIZ-B2 (United States),2009-04-21 00:00:00+00,2009-04-20 23:09:00+00,11265784,-170.217,57.15,10,10000,220.85
44,USM00070308,51,VIZ-B2 (United States),2009-04-21 00:00:00+00,2009-04-20 23:09:00+00,11265784,-170.217,57.15,10,15000,217.05
35,USM00070308,51,VIZ-B2 (United States),2009-04-21 00:00:00+00,2009-04-20 23:09:00+00,11265784,-170.217,57.15,10,20000,210.75
30,USM00070308,51,VIZ-B2 (United States),2009-04-21 00:00:00+00,2009-04-20 23:09:00+00,11265784,-170.217,57.15,10,25000,214.05
28,USM00070308,51,VIZ-B2 (United States),2009-04-21 00:00:00+00,2009-04-20 23:09:00+00,11265784,-170.217,57.15,10,30000,223.85
